In [22]:
import json
from scraper import parsuj_misje_z_url
from sqlalchemy.exc import IntegrityError
from pathlib import Path

In [23]:
url = "https://warcraft.wiki.gg/wiki/The_Hour_of_Need"
wynik = parsuj_misje_z_url(url)
wynik_json = json.dumps(wynik, ensure_ascii=False, indent=2)

In [24]:
print(wynik_json)

{
  "Źródło": {
    "url": "https://warcraft.wiki.gg/wiki/The_Hour_of_Need"
  },
  "Misje_EN": {
    "Podsumowanie_EN": {
      "Tytuł": "The Hour of Need",
      "Start_NPC": "Lor'themar Theron",
      "Koniec_NPC": "Lor'themar Theron",
      "Wymagany_Poziom": "80",
      "Doświadczenie": "1,750",
      "Nagrody": "Worn Supply Bag23 40",
      "Poprzednia_Misja": "[80] Champions of Quel'Danas,  [80] Where Heroes Hold, &  [80] My Son",
      "Następna_Misja": "[80] Luminous Wings"
    },
    "Cele_EN": {
      "Główny": {
        "1": "Rescue 20 citizens then head to the Silver Landing."
      },
      "Podrzędny": {
        "1": "Civilians Rescued (20)",
        "2": "Proceed to the Silver Landing"
      }
    },
    "Treść_EN": {
      "1": "Turalyon is thinking like a battlefield captain and not like a leader! My people, the sin'dorei, have suffered much in these past few wars. How many must die in the name of his victory?",
      "2": "Please, <name>, rescue any citizens you can a

In [25]:
from pomocnicze_funkcje_sql import *
from sqlalchemy import create_engine
from sqlalchemy.engine import URL, Engine
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError

In [26]:
zapisz_npc_i_status_do_db_z_wyniku(
    silnik=utworz_engine_do_db(),
    tabela_npc="dbo.NPC",
    tabela_npc_statusy="dbo.NPC_STATUSY",
    szukaj_wg=["Start_NPC", "Koniec_NPC"],
    wyscrapowana_tresc=wynik,
    zrodlo="wiki", # zmienic by wyszukiwalo po wzorcu
    status="0_ORYGINAŁ"
)


In [27]:
zapisz_misje_i_status_do_db_z_wyniku(
    silnik=utworz_engine_do_db(),
    wynik=wynik,
    tabela_npc="dbo.NPC",
    tabela_misje="dbo.MISJE",
    tabela_misje_statusy="dbo.MISJE_STATUSY",
    status="0_ORYGINAŁ"
)


In [28]:
# czerwony_przycisk(silnik=utworz_engine_do_db())